<a href="https://colab.research.google.com/github/ykenkou001/Pytorch-ni-yoru-hatten-deeplearning/blob/main/1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.5 「ファインチューニング」で精度向上を実現する  


*   本ファイルでは、学習ずみのVGGモデルを使用し、ファインチューニングでアリとハチの画像を分類するモデルを学習する。


# 学習目標  


1.   最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる
2.   学習したネットワークを保存・ロードできるようになる


# 事前準備  


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Pytorchによる発展ディープラーニング /\
pytorch_advanced/1_image_classification

/content/drive/MyDrive/Colab Notebooks/Pytorchによる発展ディープラーニング /pytorch_advanced/1_image_classification


In [ ]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import  models

from tqdm.notebook import  tqdm

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1235)

# DatasetとDataLoaderを作成


In [ ]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Pytorchによる発展ディープラーニング /pytorch_advanced/1_image_classification'

In [ ]:
# 1.3節で作成したクラスをフォルダutilsにあるdataloader_image_classification.pyに記載して使用
from utils.dataloader_image_classification import ImageTransform, \
make_datapath_list, HymenopteraDataset

# アリとハチの画像へのファイルパスのリストを作成
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

# Datasetを作成
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean ,std), phase='val')

# DataLoaderを作成
batch_size=32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型にオブジェクトをまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


# ネットワークを作成

In [ ]:
# 学習済みのVGG-16モデルをロード

# VGG-16モデルのインスタンスを生成
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

# VGG-16の最後の出力層の出力ユニットをアリとハチの２つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# set train mode
net.train()

print("Complete NetWork setting: set train mode and load pretrained weights")

Complete NetWork setting: set train mode and load pretrained weights


# 損失関数を定義

In [ ]:
# Setting loss function
criterion = nn.CrossEntropyLoss()

# 最適化商法を設定  
ファインチューニングは最適化手法の設定部分が転移学習と異なる。全層のパラメータを学習できるようにoptimizerを設定する。


In [ ]:
# ファインチューニングで学習させるパラメータを変数params_to_updateの１〜３に格納する

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3,weight", 
                      "classfier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# パラメータごとに各リストに格納する
for name, param in net.named_parameters():
  if update_param_names_1[0] in name:
    param.requires_grad = True
    params_to_update_1.append(param)
    print("params_to_updata_1に格納：　", name)

  elif name in update_param_names_2:
      param.requires_grad = True
      params_to_update_2.append(param)
      print("params_to_updata_2に格納：　", name)
  
  elif name in update_param_names_3:
      param.requires_grad = True
      params_to_update_3.append(param)
      print("params_to_updata_3に格納：　", name)

  else:
    param.requires_grad = False
    print("勾配計算なし。学習しない: ", name)

params_to_updata_1に格納：　 features.0.weight
params_to_updata_1に格納：　 features.0.bias
params_to_updata_1に格納：　 features.2.weight
params_to_updata_1に格納：　 features.2.bias
params_to_updata_1に格納：　 features.5.weight
params_to_updata_1に格納：　 features.5.bias
params_to_updata_1に格納：　 features.7.weight
params_to_updata_1に格納：　 features.7.bias
params_to_updata_1に格納：　 features.10.weight
params_to_updata_1に格納：　 features.10.bias
params_to_updata_1に格納：　 features.12.weight
params_to_updata_1に格納：　 features.12.bias
params_to_updata_1に格納：　 features.14.weight
params_to_updata_1に格納：　 features.14.bias
params_to_updata_1に格納：　 features.17.weight
params_to_updata_1に格納：　 features.17.bias
params_to_updata_1に格納：　 features.19.weight
params_to_updata_1に格納：　 features.19.bias
params_to_updata_1に格納：　 features.21.weight
params_to_updata_1に格納：　 features.21.bias
params_to_updata_1に格納：　 features.24.weight
params_to_updata_1に格納：　 features.24.bias
params_to_updata_1に格納：　 features.26.weight
params_to_updata_1に格納：　 features.26.bias


In [ ]:
# 最適化手法の設定
optimizer = optim.SGD([
  {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
  {'params': params_to_update_3, 'lr': 1e-3},
], momentum=0.9)

# 学習・検証を実施

In [ ]:
# Create a fuction for training model

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

  # 初期設定
  # GPUが使えるか確認
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Using Device", device)

  # ネットワークをGPUへ
  net.to(device)

  # ネットワークがある程度固定であれば、高速化させる
  torch.backends.cudnn.benchmark = True

  # loop of epoch
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-'*20)

    # epochごとの訓練と検証のループ
    for phase in ['train', 'val']:
      if phase == 'train':
        net.train() # set model to train mode
      else:
        net.eval() # set model to val mode
      
      epoch_loss = 0.0 #epochの損失和
      epoch_corrects = 0 # epochの正解和

      # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
      if (epoch == 0) and (phase == 'train'):
        continue
      
      # データローダからのミニバッチを取り出すループ
      for inputs, labels in tqdm(dataloaders_dict[phase]):

        # GPUが使えるならGPUにデータを送る
        inputs = inputs.to(device)
        labels  = labels.to(device)

        # optimizerを初期化
        optimizer.zero_grad()

        # 順伝播（foward）計算
        with torch.set_grad_enabled(phase=='train'):
          outputs = net(inputs)
          loss = criterion(outputs, labels) # compute loss
          _, preds = torch.max(outputs, 1) # predict label

          # backpropagation on training
          if phase == 'train':
            loss.backward()
            optimizer.step()

          # 結果の計算
          epoch_loss += loss.item() * inputs.size(0) # lossの合計を更新
          # 正解数の合計を更新
          epoch_corrects += torch.sum(preds == labels.data)
    
      # epochごとのlossと正解率を表示
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

In [ ]:
# 学習・検証を実行する
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Using Device cuda:0
Epoch 1/2
--------------------



val Loss: 0.7719 Acc: 0.4314
Epoch 2/2
--------------------



train Loss: 0.5109 Acc: 0.7531



val Loss: 0.1750 Acc: 0.9477


 # 学習したネットワークを保存・ロード
 

In [ ]:
# PyTorchのネットワークパラメータの保存
save_path = './weight_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [ ]:
# PyTorchのネットワークパラメータのロード
load_path = './weight_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>